In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print (tf.__version__)
print (sys.version_info)
for module in  mpl, np, pd,  tf, keras:
    print (module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

# print (train_df.head())
# print (eval_df.head())

In [3]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

# print (train_df.head())
# print (eval_df.head())
# print (y_train.head())
# print (y_eval.head())

In [4]:
# train_df.describe()

In [5]:
## feature columns：本质是规则
# 离散特征和连续特征

categorical_columns = ['sex','n_siblings_spouses','parch', 'class',
                     'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

# feature_columns本质算是一个映射规则，定义了处理特征的方式。
feature_columns =[]

for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique() #获取不重复的值
    print (categorical_column,vocab)
    
    #indicator_column用于处理成one_hot编码
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column, vocab)))
    
for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            numeric_column,dtype=tf.float32))

# croess feature: age: [1,2,3,4,5] gender:[male,female]
# age_x_gender: [(1,maile)]

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [6]:
#tf.feature_column.indicator_column?

In [7]:
def make_dataset(data_df, label_df, epochs =10, shuffle = True,
                batch_size = 32):
    # 这里将dataframe类型的dataset处理未dictionary的形式再做处理
    # Q：value的形式不可以么？
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [8]:
output_dir = "baseline_model"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# 定义estimator
baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir = output_dir,
    n_classes = 2)

baseline_estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))

W0904 21:06:46.030426 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:238: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0904 21:06:46.155498 4508796352 deprecation.py:506] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/ops/variable_scope.py:883: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0904 21:06:46.174922 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/head/base_head.py:546: to_f

In [14]:
output_dir = "linear_model"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
linear_estimator = tf.estimator.LinearClassifier(
    model_dir = output_dir,
    n_classes = 2,
    feature_columns=feature_columns)

linear_estimator.train(input_fn = lambda:make_dataset(
    train_df, y_train, epochs = 100))
    

W0904 21:11:13.190737 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/ops/lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0904 21:11:13.204242 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0904 21:11:13.205007 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_c

In [15]:
linear_estimator.evaluate(input_fn = lambda :make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

W0904 21:12:06.125063 4508796352 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


{'accuracy': 0.77272725,
 'accuracy_baseline': 0.625,
 'auc': 0.83541477,
 'auc_precision_recall': 0.79831403,
 'average_loss': 0.49247465,
 'label/mean': 0.375,
 'loss': 0.47592714,
 'precision': 0.6756757,
 'prediction/mean': 0.45297274,
 'recall': 0.75757575,
 'global_step': 1960}

In [20]:
dnn_output_dir = './dnn_model'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(
    hidden_units=[128,128],
    feature_columns=feature_columns,
    model_dir=dnn_output_dir,
    n_classes=2,
    optimizer='Adam',
    activation_fn=tf.nn.relu)
dnn_estimator.train(input_fn = lambda:make_dataset(
    train_df, y_train, epochs = 100))

W0904 21:20:33.322613 4508796352 deprecation.py:506] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1257: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
dnn_estimator.evaluate(input_fn = lambda:make_dataset(
    train_df, y_train, epochs = 1, shuffle = False))

{'accuracy': 0.8596491,
 'accuracy_baseline': 0.61244017,
 'auc': 0.9285033,
 'auc_precision_recall': 0.914621,
 'average_loss': 0.32495362,
 'label/mean': 0.3875598,
 'loss': 0.32156998,
 'precision': 0.9015544,
 'prediction/mean': 0.3453368,
 'recall': 0.7160494,
 'global_step': 1960}